oandapyをインポートし、IDとトークンを設定
口座情報などを確認する
直近5分足を取得してみる
取引コードを試してみる

In [11]:
# API接続設定のファイルを読み込む
import configparser

# 設定
config = configparser.ConfigParser()
config.read('./config_v1.txt') # パスの指定が必要です
account_ID = config['oanda']['account_id']
api_key = config['oanda']['api_key']

In [16]:
account_ID

'101-009-17484133-001'

In [17]:
api_key

'4acc6595f1e79b1dac9b723f2e1c571b-c40bdd36b16966a7c9fdfc8211546795'

In [ ]:
#!pip install oandapyV20

In [18]:
from oandapyV20 import API
from oandapyV20.exceptions import V20Error
from oandapyV20.endpoints.pricing import PricingStream
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.instruments as instruments
import json

In [19]:
#自分のアカウント、トークンをセット

api = API(access_token=api_key, environment="practice")

In [14]:
#AccountInstrumentsクラスを使って、取引可能な通貨の情報を取得します。
#paramsで対象の通貨ペアを指定してリクエストします。
params = { "instruments": "EUR_USD,EUR_JPY,USD_JPY" }
import oandapyV20.endpoints.accounts as accounts
r = accounts.AccountInstruments(accountID=accountID, params=params)
api.request(r)

NameError: name 'accountID' is not defined

In [15]:
#AccountSummaryで残高やポジション数などの口座のサマリが取得できます。
import oandapyV20.endpoints.accounts as accounts
r = accounts.AccountSummary(accountID)
api.request(r)

NameError: name 'accountID' is not defined

In [27]:
#各通貨ペアの価格データを取得するには、InstrumentsCandlesクラスを利用します。
#引数として、対象の通貨ペア（ここではUSD/JPY)と直近５件、５分足（M5)を指定しました。
import oandapyV20.endpoints.instruments as instruments
params = {
  "count": 5,
  "granularity": "M5"
}
r = instruments.InstrumentsCandles(instrument="EUR_USD", params=params)
api.request(r)

{'instrument': 'EUR_USD',
 'granularity': 'M5',
 'candles': [{'complete': True,
   'volume': 423,
   'time': '2020-12-30T08:30:00.000000000Z',
   'mid': {'o': '1.22606', 'h': '1.22611', 'l': '1.22562', 'c': '1.22573'}},
  {'complete': True,
   'volume': 372,
   'time': '2020-12-30T08:35:00.000000000Z',
   'mid': {'o': '1.22574', 'h': '1.22612', 'l': '1.22570', 'c': '1.22590'}},
  {'complete': True,
   'volume': 343,
   'time': '2020-12-30T08:40:00.000000000Z',
   'mid': {'o': '1.22592', 'h': '1.22606', 'l': '1.22576', 'c': '1.22596'}},
  {'complete': True,
   'volume': 519,
   'time': '2020-12-30T08:45:00.000000000Z',
   'mid': {'o': '1.22594', 'h': '1.22620', 'l': '1.22574', 'c': '1.22588'}},
  {'complete': False,
   'volume': 45,
   'time': '2020-12-30T08:50:00.000000000Z',
   'mid': {'o': '1.22586', 'h': '1.22600', 'l': '1.22580', 'c': '1.22600'}}]}

In [28]:
#Oanda APIではかなり複雑な注文方法にも対応しています。
#オーダーは、orders.OrderCreateでオーダーを作成してから、
#APIにリクエストすると実行されます。オーダー方法の詳細はdataパラメータで指定しています。

#ここではpriceで注文金額、有効期間としてGTC（Good unTill Cancelled : キャンセルまで有効）、
#対象の通過ペア（EUR/USD）、数量単位として100を指定しています。
#数量（units）の+はLongの買い、-はShortの売りを意味しています。
#stopLossOnFillでは、stopLossがトリガーされる金額と、
#有効期間としてGTC（Good unTill Cancelled :キャンセルまで有効）を指定しています。

#オーダー方法としては次のようなオーダーに対応しているようです。
#- MarketOrder : 成行
#- FixedPriceOrder : 指値
#- LimitOrder : 閾値より良ければ約定
#- StopOrder : 閾値より悪ければ約定

#他にもMarketIfTouchedOrder, TakeProfitOrder, StopLossOrder, TrailingStopLossOrder
#などあるようですが、使い分けはよく分かっていません。

import oandapyV20.endpoints.orders as orders
data =  {
    "order": {
        "price": "1.2",
        "stopLossOnFill": {
            "timeInForce": "GTC",
            "price": "1.22"
        },
        "timeInForce": "GTC",
        "instrument": "EUR_USD",
        "units": "-1",
        "type": "LIMIT",
        "positionFill": "DEFAULT"
    }    
}
r = orders.OrderCreate(accountID, data=data)
api.request(r)

{'orderCreateTransaction': {'id': '4',
  'accountID': '101-009-17484133-001',
  'userID': 17484133,
  'batchID': '4',
  'requestID': '24764369087183096',
  'time': '2020-12-30T08:53:14.505745139Z',
  'type': 'LIMIT_ORDER',
  'instrument': 'EUR_USD',
  'units': '-1',
  'price': '1.20000',
  'timeInForce': 'GTC',
  'triggerCondition': 'DEFAULT',
  'partialFill': 'DEFAULT',
  'positionFill': 'DEFAULT',
  'stopLossOnFill': {'price': '1.22000', 'timeInForce': 'GTC'},
  'reason': 'CLIENT_ORDER'},
 'orderCancelTransaction': {'id': '5',
  'accountID': '101-009-17484133-001',
  'userID': 17484133,
  'batchID': '4',
  'requestID': '24764369087183096',
  'time': '2020-12-30T08:53:14.505745139Z',
  'type': 'ORDER_CANCEL',
  'orderID': '4',
  'reason': 'STOP_LOSS_ON_FILL_LOSS'},
 'relatedTransactionIDs': ['4', '5'],
 'lastTransactionID': '5'}

In [31]:
#約定したオーダーのポジションを確認したりクローズすることができます。
#OpenPositionを利用して現在のポジションの一覧を取得できます。
#買い、もしくは売りのどちらかのポジションを持っていればその状況が出力されます。

#出力結果には次の情報が含まれています。
#- Long/Shortそれぞれの損益と、totalの損益
#- unrealizedPL : 現時点で未確定の損益
#- resettablePL : 確定済みの損益です。

import oandapyV20.endpoints.positions as positions
r = positions.OpenPositions(accountID=accountID)
api.request(r)

{'positions': [], 'lastTransactionID': '5'}

In [ ]:
#USD/JPYの10000のlongのポジションのうち、
#5000をクローズする場合の処理は次のようになります。

import oandapyV20.endpoints.positions as positions
data = {
  "longUnits": "5000"
}
r = positions.PositionClose(accountID=accountID,
                             instrument='USD_JPY',
                             data=data)
api.request(r)